In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from gensim.models import KeyedVectors

In [ ]:
w2v=KeyedVectors.load("/content/drive/MyDrive/FYP/CAA_NRC/Word2Vec/w2c")

In [ ]:
import pandas as pd
import numpy as np
df=pd.read_csv("/content/drive/MyDrive/FYP/CAA_NRC/CAA_NRC-preprocessed.csv")
y=pd.get_dummies(df["label"])
X=df.iloc[:,2:]


In [ ]:
maxi=-1
for i,rev in enumerate(df['new_text']):
  tokens=rev.split()
  if(len(tokens)>maxi):
    maxi=len(tokens)
print(maxi)

61


In [ ]:
import tensorflow as tf
maxlen=maxi
max_words=len(w2v.wv.vocab.keys())
from tensorflow.keras.preprocessing.sequence import pad_sequences
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_words)
def padded_sequence(X):
  tokenizer.fit_on_texts(X)
  sequences = tokenizer.texts_to_sequences(X)
  data = pad_sequences(sequences, maxlen=maxlen)
  return data
X=padded_sequence(df["new_text"])
X

array([[   0,    0,    0, ...,    7,   55,   69],
       [   0,    0,    0, ...,  139, 4955,  871],
       [   0,    0,    0, ...,   76,   81, 1784],
       ...,
       [   0,    0,    0, ...,   48,   18,    3],
       [   0,    0,    0, ...,  223,   80,   90],
       [   0,    0,    0, ...,  754,   50, 3168]], dtype=int32)

In [ ]:
import numpy as np
embed_matrix=np.zeros(shape=(max_words,300))
for word,i in tokenizer.word_index.items():
  if word in w2v:
    embed_vector=w2v[word]
  # word is in the vocabulary learned by the w2v model
    embed_matrix[i]=embed_vector

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [ ]:
embed_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.56359738,  0.01164975,  0.27649218, ...,  0.62336755,
         0.31262058,  0.24197567],
       [ 0.72490132, -0.07310981,  0.19244824, ...,  0.54099172,
         0.7085728 ,  0.35532942],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.initializers import Constant
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Dropout,Embedding,GRU,Dense,Flatten
model=Sequential()
model.add(Embedding(input_dim=max_words,output_dim=300,input_length=maxi,embeddings_initializer=Constant(embed_matrix)))
model.add(Flatten()) # loss stucks at about 
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(16,activation='relu'))
model.add(Dropout(0.60))
model.add(Dense(3,activation='softmax'))

In [ ]:
import time
start = time.time()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=1)
stop = time.time()
print(f"Training time: {stop - start}s")

Epoch 1/100
37/37 [==============================] - 1s 5ms/step - loss: 0.8614 - accuracy: 0.5553
Epoch 2/100
37/37 [==============================] - 0s 5ms/step - loss: 0.8734 - accuracy: 0.5488
Epoch 3/100
37/37 [==============================] - 0s 5ms/step - loss: 0.8619 - accuracy: 0.5578
Epoch 4/100
37/37 [==============================] - 0s 5ms/step - loss: 0.8734 - accuracy: 0.5510
Epoch 5/100
37/37 [==============================] - 0s 5ms/step - loss: 0.8664 - accuracy: 0.5639
Epoch 6/100
37/37 [==============================] - 0s 5ms/step - loss: 0.8712 - accuracy: 0.5540
Epoch 7/100
37/37 [==============================] - 0s 5ms/step - loss: 0.8638 - accuracy: 0.5617
Epoch 8/100
37/37 [==============================] - 0s 5ms/step - loss: 0.8595 - accuracy: 0.5630
Epoch 9/100
37/37 [==============================] - 0s 5ms/step - loss: 0.8652 - accuracy: 0.5613
Epoch 10/100
37/37 [==============================] - 0s 5ms/step - loss: 0.8637 - accuracy: 0.5587
Epoch 11/

In [ ]:
# y_pred=model.predict(X_test)
# for i in range(len(y_pred)):
#   if y_pred[i]>0.5:
#     y_pred[i]=1
#   else:
#     y_pred[i]=0

In [ ]:
#Metric calculation function
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
def display_metrics(y_test,y_pred):
    print(accuracy_score(y_test, y_pred))
    print(f1_score(y_test, y_pred, average="macro"))
    print(precision_score(y_test, y_pred, average="macro"))
    print(recall_score(y_test, y_pred, average="macro"))
# display_metrics(y_test,y_pred)

In [ ]:
import numpy as np
y_pred=model.predict(X_test)
y_test=y_test.to_numpy()
l=[np.argmax(i) for i in y_pred]
M=[np.argmax(i) for i in y_test]
display_metrics(M,l)

0.6572164948453608
0.6293156059150048
0.6691520962907949
0.6168100374097488
